In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request
import random
import re
 
from time import sleep
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 게시물 크롤링
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path="driver/chromedriver.exe", options = options)
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')  # user-agent 
driver.get("https://www.instagram.com/accounts/login/")
driver.maximize_window()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[5]/button/span[2]').click()
driver.execute_script('document.getElementsByName("email")[0].value = "페이스북아이디"')
driver.execute_script('document.getElementsByName("pass")[0].value = "페이스북비밀번호"')
time.sleep(1)
driver.find_element_by_xpath('//*[@id="loginbutton"]').click()
time.sleep(10)
driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()

# keyword = "바캉스"
# count = 1000
# url = "https://www.instagram.com/explore/tags/{}/".format(keyword)

def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

word = "충남여행"
url = insta_searching(word)
driver.get(url)
time.sleep(8)

def select_first(driver):
    first = driver.find_element_by_css_selector("div.eLAPa")
    first.click()
    time.sleep(8)
    
select_first(driver)

def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    return data

get_content(driver)





['"인주"세대를 아우르는 한옥의 아름다움어진 고을의 뜻을 가진 인주면에웅장한 한옥 공간이 자리해있다자칫 소홀해질 수 있는 큰 공간이섬세한 정성으로 가득하던 곳...(카페 공간이 정말 다양해서 보여드리지 못한 곳도 많아요~ 가족분들 모시고 가셔도 좋으실거 같아요 :D)-◽️카페인주◽️충남 아산시 인주면 아산만로 1608 가, 나, 다동(10:00~22:00)인주라떼 7.0-#공간노트',
 '2021-05-24',
 0,
 '충남 아산',
 ['#공간노트']]

In [3]:
# 다음페이지
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(8)

move_next(driver)

In [ ]:
# 크롤링 시작

word = "충남여행" 
url = insta_searching(word)

driver.get(url)
time.sleep(6)

select_first(driver)

results = [ ]

# 크롤링 할 페이지 개수
target = 1000
for i in range(target):
    
    try:
        data = get_content(driver)  
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

print(results[:2])

# 크롤링 결과 데이터 칼럼 설정 후 저장 (csv, json, excel)

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_csv('C:/바탕 화면/Insta_충남관광.csv')
# results_df.to_json('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata1/관광/충남2.json')
# results_df.to_excel('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata1/관광/충남관광.xlsx')
results_df

In [3]:
# 크롤링 결과 데이터 칼럼 설정 후 저장 (csv, json, excel)

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_csv('C:/바탕 화면/Insta_충남관광.csv')
# results_df.to_json('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata1/관광/충남2.json')
# results_df.to_excel('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata1/관광/충남관광.xlsx')
results_df

NameError: name 'results' is not defined

In [ ]:
# 년도별, 월별 데이터를 가져오기 위한 칼럼 설정

import pandas as pd
import re
from datetime import datetime   # 패키지 - 모듈 (이름 같음)
import pandas as pd   # csv file read
import matplotlib.pyplot as plt   # 시계열 시각화

df = pd.read_csv('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/충남관광.csv')
df['date2'] = pd.to_datetime(df['data'])
df['year'] = df['date2'].dt.year
df['month'] = df['date2'].dt.month
df

In [ ]:
# 년도별 데이터 분류

new_year = df.set_index('year')

# 2018년
year2021 = new_year.loc[[2018]]
year2021.to_excel('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/충남여행2018.xlsx')


In [ ]:
# 특정 칼럼 데이터를 index 칼럼으로 변환하여 데이터를 가져오기 위한 index 칼럼 추가

travel_insta_df = pd.DataFrame( [ ] )

folder = 'C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/'
f_list = ['충남여행.csv']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_csv(fpath)
    travel_insta_df = travel_insta_df.append(temp)

travel_insta_df.columns =['index','content','data','like','place','tags']

In [ ]:
# tags 칼럼만 수집

travel_insta_df.drop_duplicates(subset = [ "tags"] , inplace = True)
travel_insta_df.to_csv('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/충남여행.csv', index = False)

In [ ]:
# tags pandas reading

import pandas as pd
raw_total = pd.read_csv('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/충남관광.csv')
raw_total['tags']

In [ ]:
# tags 칼럼 나열

tags_total = []

for tags in raw_total['tags']:
    tags_list = tags[2:-2].split("', '")
    for tag in tags_list:
        tags_total.append(tag)
    print(tags)

In [ ]:
# counter

from collections import Counter
tag_counts = Counter(tags_total)

# 10000개
tag_counts.most_common(10000)

In [ ]:
# 전처리

STOPWORDS = ['', '#일상', '#아들스타그램', '#줌마그램', '#데일리그램', '#여섯살아들', '#도치맘', '#육아그램', '#소소한일상', 
            '#요리그램', '#홈쿡', '#얼집등원룩', '#뱀띠아들', '#6년차주부', '#daily', '#소통', '#데일리', '#2019', '#맞팔',
            '#korea', '#2020', '#좋아요', '#koreatravel', '#photography', '#소니이미지갤러리', '#snap','#육아스타그램',
            '#southkorea', '#선팔', '#2018', '#fff', '#셀스타그램', '#좋아요반사', '#ig_korea' ,'#주말일상','#딸스타그램','#스히',
            '#sony','#팔로우','#selfie', '#대한민국', '#셀카','#a7m3','#육아소통','#사랑해','#기록','#scenery','#스냅','#today',
            '#대구맘','#선팔하면맞팔','#lance_rover','#lance','#lance865','#lancetruckcamper','#truckcamper','#truckcamperlife',
            '#모터홈라이프','#motorhomelife','#알빙','#arving','#sel2470gm','#beautifuldestinations','#소통해요','#ootd','#selca', 
            '#f4f','#인친','#아들','#2015','#문경시SNS서포터즈','#igtravel','#가산수피아','#2017','#visitkorea','#instagramer',
             '#꽃비원라이프','#사춘기메들리','#Repost','#더정성스냅','#SonyImageGallery','#iphone8','#셀피','#육아맘','#백현그램',
             '#데일리룩','#igdaily','#2014','#a7ii','#selfi','#pet','#sonyphotography','#good', '#멍스타그램','#ig_asia','#좋아요',
             '#koreabyme','#dailylook','#해시태그','#오늘','#캐논이미지스토밍','#ig_color','#instadaily','#temple','#점심','#닭띠아기',
             '#예쁜아기','#귀여워','#부산사진동호회여담','#instagood','#goout','#gooutside','#꼬꼬마부부','#exploringKorea','#environment',
             '#유기견','#고령강아지', '#고령유기견','#고령요크셔테리어','#고령유기동물','#요크셔테리어','#다음은어디','#가보고싶은곳밀',
             '#sunset','#굿모닝','#2016','#2012','#ig_photo','#후추동반여행','#아들맘','#딸맘','#southkoreatrip','#southkoreatravel',
             '#2020특별여행주간','#여름원피스','#❤️','#육아소통','#travelkorea','#traveller','#리틀포레스트','#koreanlife','#likeforlikes',
             '#longexposure','#사랑','#대환투어','#대환투어on','#대환투어는계속된다','#아기일상','#baby','#cutebaby','#심장폭행',
             '#워킹맘','#직장인', '#베이비스타그램','#딸둘','#화이팅','#gbnadri','#durchatmen','#로랑_여행','#사진소통','#방콕','#외출',
             '#럽럽','#럽럽럽','#럽럽럽💕','#90mm', '#ig_world','#ig_koera','#pine', '#follow','#선팔하면맞팔가요', '#利潤益',
             '#gailbookshelf','#writeaboutmyself','#naturaldyescarf','#내고장알리미연합회','#오오티디','#following','#follower',
             '#follow4like','#likeforfollow','#안녕경주야','#portrait','#kinfolklife','#후추','#엄마와아들',
            '#충남명소','#충남여행','#충남가볼만한곳','#충남맛집','#충남데이트','#충남관광','#충남어디까지가봤니','#충남','#도민리포터',
            '#충청남도','#충남관광','#충청남도','#여행스타그램','#충남여행']

In [ ]:
# 전처리 후 데이터

tag_total_selected = []

for tag in tags_total:
    if tag not in STOPWORDS:
        tag_total_selected.append(tag)
        
# 상위 100개 단어 추출하기        
tag_counts_selected = Counter(tag_total_selected)
tag_counts_selected.most_common(100)

In [ ]:
# 그래프

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
import sys

In [ ]:
# font 설정

if sys.platform in ['win32', 'win64']:
    font_name = 'malgun gothic'
elif sys.platform == 'darwin':
    font_name = 'AppleGothic'
    
rc('font', family = font_name)

In [ ]:
# x,y 설정

tag_counts_df = pd.DataFrame(tag_counts_selected.most_common(50))
tag_counts_df.columns = ['tags', 'counts']

In [ ]:
# 데이터 막대 그래프

plt.figure(figsize =(8,10))
sns.barplot(x = 'counts', y = 'tags', data = tag_counts_df)
plt.savefig('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/충남관광2.png')  

In [ ]:
# wordcloud

! pip install wordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import platform

In [ ]:
if platform.system() == 'Windows':   
    font_path = "c:/Windows/Fonts/malgun.ttf"
elif platform.system() == "Darwin":  
    font_path = "/Users/$USER/Library/Fonts/AppleGothic.ttf"

In [ ]:
wordcloud=WordCloud(font_path= font_path, 
                    background_color="white",
                    max_words=100,
                    relative_scaling= 0.3,
                    width = 800,
                    height = 400
                 ).generate_from_frequencies(tag_counts_selected)  
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('C:/Users/82108/OneDrive/Desktop/GitHub/SpringFrameWork/Bigdata/bigdata2/data/gbtriptag-barplot4.png')  